In [ ]:
# ============================================
# 🕊 Mesa Tornado Boids 可视化（精简优化版）
# ============================================

import math, numpy as np
from typing import List,Optional
from mesa import Agent, Model
from mesa.space import ContinuousSpace,MultiGrid
from mesa.time import RandomActivation
from mesa.datacollection import DataCollector
from mesa.visualization import VisualizationElement
from mesa_viz_tornado.modules import CanvasGrid, ChartModule
from mesa.visualization.ModularVisualization import ModularServer
from mesa_viz_tornado.UserParam import Slider, NumberInput, StaticText
from mesa_viz_tornado.ModularVisualization import ModularServer as TornadoServer


# ============================================
# 1️⃣ 自动生成 JS 可视化模块
# ============================================
JS_CODE = r"""
class SimpleCanvasModule {
  constructor(canvas_width, canvas_height, space_width, space_height) {
    this.canvas_width = canvas_width;
    this.canvas_height = canvas_height;
    this.space_width = space_width;
    this.space_height = space_height;

    this.canvas = document.createElement("canvas");
    this.canvas.width = canvas_width;
    this.canvas.height = canvas_height;
    this.context = this.canvas.getContext("2d");
    document.getElementById("elements").appendChild(this.canvas);
  }

  render(data) {
    const ctx = this.context;
    ctx.clearRect(0, 0, this.canvas_width, this.canvas_height);
    for (const a of data) {
      const x = (a.x / this.space_width) * this.canvas_width;
      const y = this.canvas_height - (a.y / this.space_height) * this.canvas_height;
      ctx.beginPath();
      ctx.arc(x, y, a.r || 2, 0, 2 * Math.PI);
      ctx.fillStyle = a.Color || "#1f77b4";
      ctx.fill();
    }
  }
}
"""

with open("SimpleCanvasModule.js", "w", encoding="utf-8") as f:
    f.write(JS_CODE.strip())
print("✅ SimpleCanvasModule.js 已生成")


# ============================================
# 2️⃣ 工具函数
# ============================================
def _norm(v): return v / (np.linalg.norm(v) + 1e-12)
def _limit(v, max_mag):
    n = np.linalg.norm(v)
    return v * (max_mag / (n + 1e-12)) if n > max_mag else v


# ============================================
# 3️⃣ Boid Agent
# ============================================
class Boid(Agent):
    """连续空间中的 Boid 个体"""

    def __init__(self, unique_id, model, pos, vel,
                 max_speed, max_force, perception,
                 sep_r, w_align, w_cohere, w_separate):
        super().__init__(unique_id, model)
        self.pos = np.array(pos, dtype=float)
        self.vel = np.array(vel, dtype=float)
        self.acc = np.zeros(2)

        # 运动与行为参数
        self.max_speed = max_speed
        self.max_force = max_force
        self.perception = perception
        self.sep_r = sep_r
        self.w_align = w_align
        self.w_cohere = w_cohere
        self.w_separate = w_separate

    # ---------- 辅助函数 ----------
    def _wrap(self, pos):
        w, h = self.model.space.x_max, self.model.space.y_max
        return np.array([pos[0] % w, pos[1] % h])

    def _offset(self, a, b):
        w, h = self.model.space.x_max, self.model.space.y_max
        dx, dy = b[0] - a[0], b[1] - a[1]
        if abs(dx) > w / 2: dx -= math.copysign(w, dx)
        if abs(dy) > h / 2: dy -= math.copysign(h, dy)
        return np.array([dx, dy])

    # ---------- 行为逻辑 ----------
    def step(self):
        neighbors = self.model.space.get_neighbors(self.pos, self.perception, include_center=False)
        if not neighbors:
            self.acc[:] = 0
        else:
            avg_vel, center, sep = np.zeros(2), np.zeros(2), np.zeros(2)
            ca = cc = cs = 0
            for nb in neighbors:
                offset = self._offset(self.pos, nb.pos)
                d = np.linalg.norm(offset)
                if d < 1e-12: continue
                avg_vel += nb.vel; ca += 1
                center += nb.pos; cc += 1
                if d < self.sep_r:
                    sep -= offset / (d**2 + 1e-12); cs += 1

            steer_align = _limit(_norm(avg_vel / ca) * self.max_speed - self.vel, self.max_force) if ca > 0 else 0
            steer_cohere = _limit(_norm(self._offset(self.pos, center / cc)) * self.max_speed - self.vel, self.max_force) if cc > 0 else 0
            steer_sep = _limit(_norm(sep) * self.max_speed - self.vel, self.max_force) if cs > 0 else 0

            self.acc = _limit(
                self.w_align * steer_align +
                self.w_cohere * steer_cohere +
                self.w_separate * steer_sep,
                self.max_force
            )

        # 更新速度与位置
        self.vel += self.acc
        speed = np.linalg.norm(self.vel)
        self.vel *= np.clip(speed, 0.15 * self.max_speed, self.max_speed) / (speed + 1e-12)
        self.pos = self._wrap(self.pos + self.vel * self.model.dt)
        self.model.space.move_agent(self, tuple(self.pos))


# ============================================
# 4️⃣ Boids 模型
# ============================================
class BoidsModel(Model):
    """Boids 模型：连续空间 + 随机激活调度"""

    def __init__(self, N=150, width=12.0, height=8.0, dt=0.05,
                 perception=1.0, sep_r=0.35, max_speed=2.0, max_force=0.05,
                 w_align=1.0, w_cohere=0.8, w_separate=1.5, seed:Optional[int]=None):
        super().__init__(seed=seed)
        self.dt = dt
        self.space = ContinuousSpace(width, height, torus=True)
        self.schedule = RandomActivation(self)

        rng = np.random.default_rng(seed)
        for i in range(N):
            x, y = rng.uniform(0, width), rng.uniform(0, height)
            heading = rng.uniform(0, 2 * np.pi)
            speed = rng.uniform(0.3 * max_speed, 0.8 * max_speed)
            vel = np.array([math.cos(heading), math.sin(heading)]) * speed
            agent = Boid(i, self, (x, y), vel, max_speed, max_force,
                         perception, sep_r, w_align, w_cohere, w_separate)
            self.space.place_agent(agent, (x, y))
            self.schedule.add(agent)

    def step(self):
        self.schedule.step()


# ============================================
# 5️⃣ Tornado 可视化模块
# ============================================
class SimpleContinuousModule(VisualizationElement):
    local_includes = ["SimpleCanvasModule.js"]

    def __init__(self, portrayal_method, canvas_w, canvas_h, space_w, space_h):
        super().__init__()
        self.portrayal_method = portrayal_method
        self.js_code = f"elements.push(new SimpleCanvasModule({canvas_w}, {canvas_h}, {space_w}, {space_h}));"
        self.canvas_w, self.canvas_h, self.space_w, self.space_h = canvas_w, canvas_h, space_w, space_h
        self._last_reset = None

    def render(self, model):
        rid = getattr(model, "reset_id", None)
        if rid is not None and rid != self._last_reset:
            self._last_reset = rid
            return []  # 清屏帧
        return [
            dict(x=float(a.pos[0]), y=float(a.pos[1]), **self.portrayal_method(a))
            for a in model.schedule.agents
        ]


def boid_portrayal(agent):
    return {"Color": "#1f77b4", "r": 2}


canvas_element = SimpleContinuousModule(boid_portrayal, 720, 480, 12, 8)


# ============================================
# 6️⃣ 参数与服务器
# ============================================
model_params = {
    "N": Slider("Boids 数量", 80, 10, 500, 10),
    "dt": Slider("时间步长", 0.05, 0.01, 0.2, 0.01),
    "perception": Slider("感知半径", 1.0, 0.2, 3.0, 0.1),
    "sep_r": Slider("分离半径", 0.35, 0.1, 1.5, 0.05),
    "max_speed": Slider("最大速度", 2.0, 0.5, 6.0, 0.1),
    "max_force": Slider("最大转向", 0.05, 0.01, 0.5, 0.01),
    "w_align": Slider("对齐权重", 1.0, 0.0, 3.0, 0.1),
    "w_cohere": Slider("聚合权重", 0.8, 0.0, 3.0, 0.1),
    "w_separate": Slider("分离权重", 1.5, 0.0, 3.0, 0.1),
}


# ✅ 自定义可重置服务器（支持重新启动模型）
class ResettableServer(TornadoServer):
    def reset_model(self):
        params = {k: (v.value if hasattr(v, "value") else v) for k, v in self.model_kwargs.items()}
        self.model = self.model_cls(**params)
        self.model.running = True
        print("✅ 模型已重置：", params)


# ============================================
# 7️⃣ 启动模拟
# ============================================
if __name__ == "__main__":
    server = ModularServer(
        BoidsModel,
        [canvas_element],
        "Boids Tornado 模拟（精简版）",
        model_params,
    )
    server.port = 8535
    server.launch()


✅ SimpleCanvasModule.js 已生成
Interface starting at http://127.0.0.1:8534


C:\Users\Saru\AppData\Local\Temp\ipykernel_44028\3461745555.py:158: UserWarning: Agent 0 is being placed with
place_agent() despite already having the position [2.63972479 1.22350613]. In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.space.place_agent(agent, (x, y))
C:\Users\Saru\AppData\Local\Temp\ipykernel_44028\3461745555.py:158: UserWarning: Agent 1 is being placed with
place_agent() despite already having the position [9.63140533 6.94241352]. In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.space.place_agent(agent, (x, y))
C:\Users\Saru\AppData\Local\Temp\ipykernel_44028\3461745555.py:158: UserWarning: Agent 2 is being placed with
place_agent() despite already having the position [8.02044735 0.46045837]. In most
cases, you'd want to clear the current position with remove_agent()
before placing the agent again.
  self.space.place_agent(agent, (x, y))
C

RuntimeError: This event loop is already running

Socket opened!
{"type":"get_step","step":1}
{"type":"get_step","step":2}
{"type":"get_step","step":3}
{"type":"get_step","step":4}
{"type":"get_step","step":5}
{"type":"get_step","step":6}
{"type":"get_step","step":7}
{"type":"get_step","step":8}
{"type":"get_step","step":9}
{"type":"get_step","step":10}
{"type":"get_step","step":11}
{"type":"get_step","step":12}
{"type":"get_step","step":13}
{"type":"get_step","step":14}
{"type":"get_step","step":15}
{"type":"get_step","step":16}
{"type":"get_step","step":17}
{"type":"get_step","step":18}
{"type":"get_step","step":19}
{"type":"get_step","step":20}
{"type":"get_step","step":21}
{"type":"get_step","step":22}
{"type":"get_step","step":23}
{"type":"get_step","step":24}
{"type":"get_step","step":25}
{"type":"get_step","step":26}
{"type":"get_step","step":27}
{"type":"get_step","step":28}
{"type":"get_step","step":29}
{"type":"get_step","step":30}
{"type":"get_step","step":31}
{"type":"get_step","step":32}
{"type":"get_step","step":33}
{"ty